<a href="https://colab.research.google.com/github/IngbA/ETL_PYTHON/blob/main/SantanderDevWeek2023_NLTK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [ ]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [ ]:
import pandas as pd
import requests
import json

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[1, 2, 3, 4, 5]


#Configurando a API para uso do NLTK

In [ ]:
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

#Leitura dos IDs

In [ ]:
def ler_ids_usuarios():
  df = pd.read_csv('SDW2023.csv')
  user_ids = df['UserID'].tolist()
  print(f'Ids de Usuario lidos: {user_ids}')
  return user_ids

print(df)

   UserID
0       1
1       2
2       3
3       4
4       5


#Obtendo dados de usuario pela API

In [ ]:
def obter_dados_usuario(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := obter_dados_usuario(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 1,
    "name": "Devweekerson",
    "account": {
      "id": 1,
      "number": "01.097954-4",
      "agency": "2030",
      "balance": 624.12,
      "limit": 1000.0
    },
    "card": {
      "id": 1,
      "number": "xxxx xxxx xxxx 1111",
      "limit": 2000.0
    },
    "features": [
      {
        "id": 1,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pix.svg",
        "description": "PIX"
      },
      {
        "id": 2,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pay.svg",
        "description": "Pagar"
      },
      {
        "id": 3,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/transfer.svg",
        "description": "Transferir"
      },
      {
        "id": 4,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg",
        "description": "Conta Corrente"
      },
      {
        

#Geracao de Mensagens de Marketing Personalizadas

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Função para gerar mensagem de marketing com base em regras
def gerar_mensagem_mkt(user):
    message = f"Olá {user['name']}! Acreditamos que investir é o caminho para o sucesso financeiro. \
                \nVenha descobrir nossos produtos e comece a investir hoje mesmo!"
    return message

# Supondo que você tenha uma lista de usuários chamada 'users'
for user in users:
    nova_mensagem = gerar_mensagem_mkt(user)
    print(nova_mensagem)
    user['news'].append({
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": nova_mensagem
    })

Olá Devweekerson! Acreditamos que investir é o caminho para o sucesso financeiro.                 
Venha descobrir nossos produtos e comece a investir hoje mesmo!


#Atualizacao de novos usurarios

In [ ]:
def atualizar_novos_usuarios(user):
    response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
    return True if response.status_code == 200 else False

def execucao_completa():
    user_ids = ler_ids_usuarios()
    users = [user for id in user_ids if (user := obter_dados_usuario(id)) is not None]

    for user in users:
        nova_mensagem = gerar_mensagem_mkt(user)
        print(nova_mensagem)
        user['news'].append({
            "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
            "description": nova_mensagem
        })

    for user in users:
        success = atualizar_novos_usuarios(user)
        print(f"Usuario {user['name']} atualizado? {success}!")

execucao_completa()

Ids de Usuario lidos: [1, 2, 3, 4, 5]
Olá Devweekerson! Acreditamos que investir é o caminho para o sucesso financeiro.                 
Venha descobrir nossos produtos e comece a investir hoje mesmo!
Usuario Devweekerson atualizado? False!


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [ ]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Devweekerson updated? False!
